In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df_path = "/kaggle/input/tabular-playground-series-feb-2021/train.csv"
test_df_path = "/kaggle/input/tabular-playground-series-feb-2021/test.csv"

# Introduction
Here I will guide you through how to make a very simple baseline with PyCaret. (https://pycaret.org/) 


In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
from pycaret.regression import *

# Load and take a look at the data

In [ ]:
df = pd.read_csv(train_df_path)

In [ ]:
df.head()

In [ ]:
df.columns

We can guest (or you did read the data description) that `cat*` are categorical data, and `cont*` are numerical data. We will tell PyCaret that, ignore `id` column and take `target` as label. 

# Start modeling

## Set up experiment 

In [ ]:
cat_col = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7','cat8', 'cat9']
cont_col = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5','cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12','cont13']

In [ ]:
exp_clf = setup(data = df, target = 'target',  session_id=123, 
                numeric_features=cont_col, 
                categorical_features=cat_col,
               ignore_features=["id"], silent=True)

## Compare some modeling algorithms and decide which one to use

In [ ]:
best_model = compare_models(include=["br", "lr", "ridge"], sort="RMSE")


I put arg `include=["br", "lr", "ridge"]` to tell PyCaret only try on Bayesian Ridge, Linear Regression and Ridge Regression because I know they are super fast, and I want my baseline done fast. 

You can pass `include=None` and `turbo=False` to give it a try on all suported model. It will take a long time. 

In [ ]:
# here is list of supported model 
models()

## Tune our best model 

In [ ]:
tuned_model = tune_model(best_model, optimize = 'RMSE')

##  Finalize model
PyCaret will train your tuned model, with best hyperparameter on full train dataset (include the split for validation)

In [ ]:
final_model = finalize_model(tuned_model)

# Predict on test data 

In [ ]:
df_test = pd.read_csv(test_df_path)

In [ ]:
df_test.head()

In [ ]:
y_pred = predict_model(final_model, df_test)

In [ ]:
y_pred.head()

# Format data for submission

In [ ]:
df_result = df_test[["id"]]
df_result["target"] = y_pred["Label"]

In [ ]:
df_result

In [ ]:
df_result.to_csv('submission.csv', index=False)